In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df=pd.read_csv('https://raw.githubusercontent.com/mikdadkanbar/HousePricesML/main/data_cleaned.csv')
df.head() 


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,MoSold,YrSold,SaleType,SaleCondition,SalePrice,RoomsAndArea
0,60,RL,65.0,8450,Pave,Reg,Lvl,Inside,Gtl,CollgCr,...,0,61,0,0,2,2008,WD,Normal,208500,13680
1,20,RL,80.0,9600,Pave,Reg,Lvl,FR2,Gtl,Veenker,...,298,0,0,0,5,2007,WD,Normal,181500,7572
2,60,RL,68.0,11250,Pave,IR1,Lvl,Inside,Gtl,CollgCr,...,0,42,0,0,9,2008,WD,Normal,223500,10716
3,70,RL,60.0,9550,Pave,IR1,Lvl,Corner,Gtl,Crawfor,...,0,35,272,0,2,2006,WD,Abnorml,140000,12019
4,60,RL,84.0,14260,Pave,IR1,Lvl,FR2,Gtl,NoRidge,...,192,84,0,0,12,2008,WD,Normal,250000,19782
